In [ ]:
pip install tabula-py

In [ ]:
pip install jpype1

In [ ]:
import pandas as pd
import tabula
import re
import os

In [ ]:
note_data = []

for filename in os.listdir():
  if os.path.isfile(filename):
    print(filename)
    data = tabula.read_pdf(filename, multiple_tables=False, pages='all', stream=True, guess=False)
    df = data[0]
    n_columns = len(df.columns)

    if n_columns == 5:

      df.rename(columns={df.columns[0]: 'BOVESPA', df.columns[1]: 'NOTA', df.columns[2]: 'PRECO', df.columns[3]: 'DATA', df.columns[4]: 'NADA'}, inplace=True)

      if isinstance(df['DATA'].iloc[1], str):
        date = df['DATA'].iloc[1].strip()
      elif isinstance(df['NADA'].iloc[1], str):
        date = df['NADA'].iloc[1].strip()
      else:
        date = df['Unnamed: 5'].iloc[1].strip()

      operations = list(df[df['BOVESPA'].str.contains("1-BOVESPA",na=False)].index)

      for current_row in operations:
        #include here the steps to get the buy/sell operation
        cell_value = df['BOVESPA'].iloc[current_row] # NOTA DE CORRETAGEM ANTIGA
        c_vRegex = re.compile(r'(?<=\s)[a-zA-Z]\s*') # NOTA DE CORRETAGEM ANTIGA
        match = c_vRegex.search(cell_value)
        if match:
          c_v = match.group(0).strip()
        else:
          cell_value = df['NOTA'].iloc[current_row] # NOTA DE CORRETAGEM NOVA
          c_v = cell_value[0] # NOTA DE CORRETAGEM NOVA

        #include here the steps to get the stock title
        cell_value = df['NOTA'].iloc[current_row]
        if 'FRACIONARIO' in cell_value:
          stock_titleRegex = re.compile(r'(?<=FRACIONARIO\s)(.*)(?=\s\d+)')
        elif 'VISTA' in cell_value:
          stock_titleRegex = re.compile(r'(?<=VISTA\s)(.*)(?=\s\d+)')
        stock_title = stock_titleRegex.search(cell_value).group(0).strip()

        #include here the steps to get price
        price = df['PRECO'].iloc[current_row].strip()
        price_float = float(price.replace('.','').replace(',','.'))

        #include here the steps to get the quantity
        cell_value = df['NOTA'].iloc[current_row]
        quantityRegex = re.compile(r'(?<=\s)\d*\s*$')
        match = quantityRegex.search(cell_value)
        if match:
          quantity = quantityRegex.search(cell_value).group(0).strip()
        else:
          quantity = filename

        row_data = [date, c_v, stock_title, price, quantity]
        note_data.append(row_data)

    if n_columns == 6:

      df.rename(columns={df.columns[0]: 'BOVESPA', df.columns[1]: 'COMPRAVENDA', df.columns[2]: 'NOTA', df.columns[3]: 'PRECO', df.columns[4]: 'DATA'}, inplace=True)

      if isinstance(df['DATA'].iloc[1], str):
        date = df['DATA'].iloc[1].strip()
      else:
        date = df['Unnamed: 5'].iloc[1].strip()

      operations = list(df[df['BOVESPA'].str.contains("1-BOVESPA",na=False)].index)

      for current_row in operations:
        #include here the steps to get the buy/sell operation
        cell_value = df['BOVESPA'].iloc[current_row] # NOTA DE CORRETAGEM ANTIGA
        c_vRegex = re.compile(r'(?<=\s)[a-zA-Z]\s*') # NOTA DE CORRETAGEM ANTIGA
        match = c_vRegex.search(cell_value)
        if match:
          c_v = match.group(0).strip()
        else:
          cell_value = df['COMPRAVENDA'].iloc[current_row] # NOTA DE CORRETAGEM NOVA
          c_v = cell_value[0] # NOTA DE CORRETAGEM NOVA

        #include here the steps to get the stock title
        cell_value = df['NOTA'].iloc[current_row]
        if 'FRACIONARIO' in cell_value:
          stock_titleRegex = re.compile(r'(?<=FRACIONARIO\s)(.*)(?=\s\d+)')
        elif 'VISTA' in cell_value:
          stock_titleRegex = re.compile(r'(?<=VISTA\s)(.*)(?=\s\d+)')
        stock_title = stock_titleRegex.search(cell_value).group(0).strip()

        #include here the steps to get price
        price = df['PRECO'].iloc[current_row].strip()
        price_float = float(price.replace('.','').replace(',','.'))

        #include here the steps to get the quantity
        cell_value = df['NOTA'].iloc[current_row]
        quantityRegex = re.compile(r'(?<=\s)\d*\s*$')
        match = quantityRegex.search(cell_value)
        if match:
          quantity = quantityRegex.search(cell_value).group(0).strip()
        else:
          quantity = filename

        row_data = [date, c_v, stock_title, price, quantity]
        note_data.append(row_data)

cols = ['Data', 'Compra/Venda', 'Ticker', 'Preço', 'Quantidade']
note_df = pd.DataFrame(data=note_data, columns=cols)
note_df.to_excel("extrato_notas_corretagem_todas_2023.xlsx")
